In [1]:
from pathlib import Path
import dask.dataframe as dd
import numpy as np
import datetime
import pandas as pd
from scipy import stats 

In [2]:
# loading metadata file
filePath = 'filePath'
df = pd.read_csv(filePath + '.csv').drop_duplicates()
# Pre-processing to filter out inconsistent data
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]

104692
104692


In [3]:
lista = []
# List all files in directory using pathlib
basepath = Path("path")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)

In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file
df_in_2019 = df[df.Identifier.isin(lista_FileName)]

In [5]:
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]

## Selection of Identifier with certains characteristics

In [6]:
df_in_2019= df_in_2019[(df_in_2019['Country']=='US') | (df_in_2019['Country']=='CA')]
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 2 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df3 = df_in_2019[(df_in_2019['Number of Occupants'] == 3 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df4 = df_in_2019[(df_in_2019['Number of Occupants'] == 4 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
No_RS=3
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(916, 22)
(3802, 22)
(2418, 22)
(2813, 22)


In [7]:
use_cols = [cols]
mainPath ='mainPath'

## Drop repeated DateTime of each Identifier, mean activation = activation/ number of sensors with value at that time

In [ ]:
##### DF1

# Take all Identifiers of df, to after open them
a1=df1.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a1:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
    df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

    df_motion['DateTime'] = df['DateTime']
    df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

    df_motion.drop_duplicates('DateTime', inplace=True)
    df_motion[['DateTime','mean_activation']].to_csv('path'+filename+'.csv')


In [79]:
##### DF2

# Take all Identifiers of df, to after open them
a2=df2.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a2:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
    df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

    df_motion['DateTime'] = df['DateTime']
    df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

    df_motion.drop_duplicates('DateTime', inplace=True)
    df_motion[['DateTime','mean_activation']].to_csv('path/'+filename+'.csv')

In [80]:
##### DF3

# Take all Identifiers of df, to after open them
a3=df3.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a3:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
    df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

    df_motion['DateTime'] = df['DateTime']
    df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

    df_motion.drop_duplicates('DateTime', inplace=True)
    df_motion[['DateTime','mean_activation']].to_csv('path'+filename+'.csv')

In [8]:
##### DF4

# Take all Identifiers of df, to after open them
a4=df4.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a4:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
    df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

    df_motion['DateTime'] = df['DateTime']
    df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

    df_motion.drop_duplicates('DateTime', inplace=True)
    df_motion[['DateTime','mean_activation']].to_csv('path'+filename+'.csv')

## Mean value for Identifiers with 1,2,3,4 occup

In [17]:
ff=dd.read_csv('path/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('path/', '')
ff['path']=ff['path'].str.replace('.csv', '')
ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df1[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll=result_g.compute()
ll_compu=pd.DataFrame(ll.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')"]].plot(figsize=(25,8))

In [20]:
ff=dd.read_csv('path/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('path/', '')
ff['path']=ff['path'].str.replace('.csv', '')
ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll2=result_g.compute()
ll_compu=pd.DataFrame(ll2.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')"]].plot(figsize=(25,8))

In [22]:
ff=dd.read_csv('/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')
ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df3[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll3=result_g.compute()
ll_compu=pd.DataFrame(ll3.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')"]].plot(figsize=(25,8))

In [24]:
ff=dd.read_csv('/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')
ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df4[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll4=result_g.compute()
ll_compu=pd.DataFrame(ll4.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')"]].plot(figsize=(25,8))

## filter ID with less than 245 days with information (245*24*12 cant samples each 5 min in 245 days) 

In [27]:
##### DF1

# Take all Identifiers of df, to after open them
a1=df1.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a1:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    if  len(df_motion[df_motion['working_sensors'] > 3]) >= 245*24*12:

        df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
        df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

        df_motion['DateTime'] = df['DateTime']
        df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

        df_motion.drop_duplicates('DateTime', inplace=True)
        df_motion[['DateTime','mean_activation']].to_csv('/'+filename+'.csv')
    

In [36]:
ff=dd.read_csv(/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')
ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df1[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll_1=result_g.compute()
ll_compu=pd.DataFrame(ll_1.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')"]].plot(figsize=(25,8))

In [45]:
##### DF2

# Take all Identifiers of df, to after open them
a2=df2.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a2:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    if  len(df_motion[df_motion['working_sensors'] > 3]) >= 245*24*12:

        df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
        df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

        df_motion['DateTime'] = df['DateTime']
        df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

        df_motion.drop_duplicates('DateTime', inplace=True)
        df_motion[['DateTime','mean_activation']].to_csv('/'+filename+'.csv')
    


In [48]:
ff=dd.read_csv('/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')

ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll_2=result_g.compute()
ll_compu=pd.DataFrame(ll_2.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/1000 
ll_compu[["('mean_activation', 'mean')","('mean_activation', 'count')"]].plot(figsize=(25,8))

## Take only dateTimes with information from at least 4 sensors

In [21]:
##### DF1

# Take all Identifiers of df, to after open them
a1=df1.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a1:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    if  len(df_motion[df_motion['working_sensors'] > 3]) >= 245*24*12:

        df_motion = df_motion[df_motion['working_sensors'] > 3] # Take only dateTimes with information from at least 4 sensors
        df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
        df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 

        df_motion['DateTime'] = df['DateTime']
        df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])

        df_motion.drop_duplicates('DateTime', inplace=True)
        df_motion[['DateTime','mean_activation']].to_csv('/'+filename+'.csv')

In [14]:
ff=dd.read_csv('/*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')

ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df1[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll_1_1=result_g.compute()


In [15]:
pd.DataFrame(ll_1_1.to_records()).set_index('DateTime').to_csv('')
ll_compu=pd.read_csv('').set_index('DateTime') 
ax=ll_compu[["('mean_activation', 'count')"]].plot(figsize=(20,6))
ax.legend(['Number of Identifiers'], fontsize=16)
ax.set_title('1 occupant at least 245 days, DateTime at least from 4 sensors',fontsize=16)


# Filter days that more than 70% identifiers have no data

In [104]:

ll_1_1=pd.read_csv('')
ll_compu_1 = pd.DataFrame(ll_1_1.to_records()).set_index('DateTime').drop('index',axis=1)
days_nan1 = pd.DataFrame(pd.to_datetime(ll_compu_1[ll_compu_1["('mean_activation', 'count')"] > ((ll_compu_1[["('mean_activation', 'count')"]])*0.7).max().iloc[0]].reset_index()['DateTime']))

In [110]:
##### DF1
# Take all Identifiers of df, to after open them
a1=df1.loc[:,'Identifier'].tolist()
path=mainPath

for filename in a1:
    df = pd.read_csv(mainPath+filename+'.csv')

    df_motion = df.loc[:, df.columns.str.contains('_Motion')]
    df_motion['working_sensors'] = 11-df_motion.isna().sum(axis=1)

    if  len(df_motion[df_motion['working_sensors'] > 3]) >= 245*24*12:
        pp1=df_motion
        df_motion = df_motion[df_motion['working_sensors'] > 3] # Take only dateTimes with information from at least 4 sensors
        df_motion['activation'] = df_motion.loc[:, df_motion.columns.str.contains('_Motion')].sum(axis=1)
        df_motion['mean_activation'] = df_motion['activation']/df_motion['working_sensors'] 
        
        df_motion['DateTime'] = df['DateTime']
        df_motion['DateTime'] = pd.to_datetime(df_motion['DateTime'])
        df_motion.drop_duplicates('DateTime', inplace=True)
        
        df_motion = pd.merge(df_motion, days_nan1['DateTime'], on='DateTime')
        
        df_motion[['DateTime','mean_activation']].to_csv('/'+filename+'.csv')

In [10]:
ff=dd.read_csv('*', parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')

ff_columns = ['path', 'mean_activation', 'DateTime']
result = dd.merge(df1[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result_g = result[['mean_activation','DateTime']].groupby(['DateTime']).agg(['mean', 'std', 'count'])
ll_1_1_filtro=result_g.compute()

ll_compu=pd.DataFrame(ll_1_1_filtro.to_records()).set_index('DateTime')
ll_compu[["('mean_activation', 'count')"]]=ll_compu[["('mean_activation', 'count')"]]/np.max(ll_compu[["('mean_activation', 'count')"]]) 
ll_compu[["('mean_activation', 'mean')","('mean_activation', 'count')"]].plot(figsize=(25,8), title='1 occupant at least 245 days, DateTime at least from 4 sensors, 70% of identifiers', color=['#1f77b4','g'])


#  filtering by percentiles

In [59]:
from pathlib import Path
lista1 = []
# List all files in directory using pathlib
basepath = Path("_1/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista1.append(item.name)
lista_FileName1 = [s.replace('.csv', '') for s in lista1]

In [60]:
path_one = '1/' # use your path
all_files_read1 = []

for filename in lista_FileName1:
    df_p = pd.read_csv(path_one+filename+'.csv', index_col=None, header=0)
    
    df_p['date'] = pd.to_datetime(df_p['DateTime']).dt.date
    df_p = df_p[['date','mean_activation']].groupby('date').mean()
    df_p['path'] = filename
    all_files_read1.append(df_p)
    
frame_all_files1 = pd.concat(all_files_read1, axis=0, ignore_index=False)

In [62]:
frame_all_files1.to_csv('es.csv')

In [17]:
frame_all_files1=pd.read_csv('es.csv')
percentile_min = frame_all_files1[['date','mean_activation']].groupby('date').agg(lambda x: np.percentile(x, 20))
percentile_min = pd.DataFrame(percentile_min)
percentile_max =frame_all_files1[['date','mean_activation']].groupby('date').agg(lambda x: np.percentile(x, 80))
percentile_max=pd.DataFrame(percentile_max)

In [18]:
group11 = frame_all_files1[['date','mean_activation','path']].groupby('path')
li11_percentile = []
complete=0
incomplete=0
for key, g_dataframe in group11:
    if len(g_dataframe)>1:# take off this condition
        fds_max=(percentile_max.reset_index()[['mean_activation','date']].merge(g_dataframe[['mean_activation','date']], how='outer', on='date',suffixes=('_percentilemax', '_df')))
        fds_max_min=(percentile_min.reset_index()[['mean_activation','date']].merge(fds_max[['mean_activation_percentilemax','date','mean_activation_df']], how='outer', on='date'))
        df_per_minmmax = fds_max_min[(fds_max_min['mean_activation_percentilemax']>fds_max_min['mean_activation_df']) & (fds_max_min['mean_activation']<fds_max_min['mean_activation_df'])]# 
       
        if len(df_per_minmmax)>200:
            complete=complete+1
            li11_percentile.append(g_dataframe['path'].iloc[0])
            
        else:
            incomplete=incomplete+1

In [20]:
pd.DataFrame(li11_percentile).to_csv('csv')

In [11]:
group11 = frame11[['DateTime','activation','path']].groupby('path')
li11_percentile = []
complete=0
incomplete=0
for key, g_dataframe in group11:
    if len(g_dataframe)>200:

        fds_max=(percentile_max.reset_index()[['activation','DateTime']].merge(g_dataframe[['activation','DateTime']], how='outer', on='DateTime',suffixes=('_percentilemax', '_df')))
        fds_max_min=(percentile_min.reset_index()[['activation','DateTime']].merge(fds_max[['activation_percentilemax','DateTime','activation_df']], how='outer', on='DateTime'))
        df_per_minmmax = fds_max_min[(fds_max_min['activation_percentilemax']>fds_max_min['activation_df']) & (fds_max_min['activation']<fds_max_min['activation_df'])]# 
       
        if len(df_per_minmmax)>150:
            complete=complete+1
            li11_percentile.append(g_dataframe)
        else:
            incomplete=incomplete+1
       
print(complete)
print(incomplete)
frame111 = pd.concat(li11_percentile,  ignore_index=False)
frame111=frame111.reset_index()
ax1 = frame111[['DateTime','activation']].groupby('DateTime').mean().plot(figsize=(25,8),title='Identifiers with 1 occupant,4 sensors, from US')
frame111[['DateTime','activation']].groupby('DateTime').std().plot(ax=ax1)
ax1.set_ylabel('10% trimmed mean of sum sensors values')

466
56
